In [1]:
import random
import statistics as st
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import pandas_datareader.data as reader
import datetime as dt
import matplotlib.pyplot as plt
import getFamaFrenchFactors as gff
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import numpy_financial as npf
import plotly.io as pio


<h1><center>Overview</center></h1>

### <p4><center>Cost of Equity:        Fama French Three Factor Model</center></p4>
### <p4><center>Valuation:             Multi-Stage Dividend Discount Model</center></p4>
### <p4><center>Sensitivity Analysis:  Monte Carlo Simulations</center></p4>
#### <p4><center>Company: Enbridge Inc</center></p4>
#### <p4><center>Analyst: Omar Shehata</center></p4>
#### <p4><center>Sector Manager: Aryan Falke</center></p4>
#### <p3><center>Prepared for: The Fund</center></p3>


In [2]:
end = dt.datetime.now()
start = dt.date(end.year-11, end.month, end.day)
ticker= ['ENB.TO']
ENB = yf.Ticker(ticker[0])
div = ENB.dividends[-1]*4


In [3]:
div_history=pd.DataFrame(ENB.dividends)
div_history["Growth"] = (div_history["Dividends"] - div_history["Dividends"].shift()) / abs(div_history["Dividends"].shift())
yearly_growth=div_history[(div_history != 0).all(1)]
gr_yearly= yearly_growth['Growth']
div_yearly = yearly_growth['Dividends']
index= yearly_growth.index
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)
fig=fig.add_trace(go.Scatter(x=index, y=gr_yearly, name='Growth Rate %'),
              row=2, col=1)

fig=fig.add_trace(go.Scatter(x=index, y=div_yearly, name = 'Dividend (CAD)'),
              row=1, col=1)

fig=fig.update_layout(height=600, width=900,
                  title_text="Quarterly Dividend and Growth Rate")


### Three Factor Model (Fama-Fremch, 1993)

>$$R_{s}-R_{f}=\alpha+\beta_{mkt}\left(R_{mkt}-R_{f}\right)+\beta_{SMB}R_{SMB}+\beta_{HML}R_{HML}$$

where: <br>
$R_{s}-R_{f}$ = cost of equity <br>
$R_{s}$ = return on stock <br>
$R_{f}$ = risk-free rate <br>
$\beta_{mkt}$ = market beta <br>
$R_{mkt}$ = market premuim<br>
$\beta_{SMB}$ = size beta <br>
$R_{SMB}$ = size premuim<br>
$\beta_{HML}$ = value beta <br>
$R_{HML}$ = value premuim <br>

*Model was chosen instead of traditional CAPM due to the stock's low Beta.

Get Monthly Fama-french 3 Factors from Kenneth R. French  data library And Historical Returns

In [4]:
ticker_us=['ENB']
stock_prices = reader.get_data_yahoo(ticker_us, start, end)['Adj Close']
stock_prices = stock_prices.resample('1M').last()
stock_returns = stock_prices.pct_change()
stock_returns = stock_returns.dropna()
ff3_monthly = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
ff3_monthly.rename(columns={'date_ff_factors':'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace = True)
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()
data= ff3_monthly.merge(stock_returns, on = 'Date')
data.head()
data = data.tail(60)
data.shape
excess_return = data['ENB']-data['RF']
data['ENB.TO'+'-RF']=excess_return

In [5]:
data.tail()

,Mkt-RF,SMB,HML,RF,ENB,ENB.TO-RF
Date,,,,,,
2022-06-30,-0.0843,0.0209,-0.0597,0.0006,-0.084291,-0.084891
2022-07-31,0.0957,0.0281,-0.0410,0.0008,0.063181,0.062381
2022-08-31,-0.0377,0.0139,0.0031,0.0019,-0.068393,-0.070293
2022-09-30,-0.0935,-0.0082,0.0003,0.0019,-0.099952,-0.101852
2022-10-31,0.0783,0.0010,0.0806,0.0023,0.049865,0.047565


## OLS Regression to Estimate Beta

In [6]:
X =  data[['Mkt-RF', 'SMB', 'HML']]
Y = data[ticker[0]+'-RF']

X1 = sm.add_constant(X)
model = sm.OLS(Y,X1)
results = model.fit()

intercept, beta_m, beta_s, beta_v = results.params


In [7]:
print('Market Beta:',beta_m, 'Size Beta:', beta_s, 'Value Beta', beta_v)
results.summary()


Market Beta: 0.7469595281012666 Size Beta: 0.21813642536996913 Value Beta 0.4791348969861034


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              ENB.TO-RF   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     18.38
Date:                Fri, 25 Nov 2022   Prob (F-statistic):           2.01e-08
Time:                        05:50:42   Log-Likelihood:                 95.829
No. Observations:                  60   AIC:                            -183.7
Df Residuals:                      56   BIC:                            -175.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.007      0.076      0.939      -0.013       0.014
Mkt-RF         0.7470      0.126      5.913      0.000       0.494       1.000
SMB            0.2181      0.255      0.856      0.396      -0.292       0.729
HML            0.4791      0.150      3.203      0.002       0.179       0.779
==============================================================================
Omnibus:                        1.942   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.379   Jarque-Bera (JB):                1.235
Skew:                           0.317   Prob(JB):                        0.539
Kurtosis:                       3.303   Cond. No.                         39.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Calculate Cost of Equity

In [8]:
risk_free = data['RF'].mean()
risk_free
exp_return = risk_free + beta_m*market_premium + beta_s*size_premium + beta_v*value_premium
cost_capital=exp_return*12 
print(round(cost_capital*100,2),"%")

9.66 %


## Three Stage Dividend Discount Model
>$$P=\frac{D_{1}}{(R+1)^1}+\frac{D_{2}}{(R+1)^2}+ ... +\frac{D_{N}}{(R+1)^N}+\frac{D_{N}(1+G2)+D_{N}H(G1-G2)}{\frac{(R-G2)}{(R+1)^N}}$$

where: <br> 
$P$ = price <br>
$D_{1}$ = value of next year dividend <br>
$R$ = constant cost of equity capital <br>
$N$ = period <br>
$G1$ = High growth rate <br>
$G2$ = Transitional growth rate<br>
$G3$ = constant growth rate in perpetuity <br>


Assumptions: <br> 
$N$ = 5 <br>
$G1$ = 10Y Historical Div Growth (mean: 10%, std: 8.5%) <br>
$G2$ = 5Y Historical Div Growth (mean: 7%, std: 3.7%)<br>
$G3$ = mean: 2%, std: 0.5% <br>
$Beta$ = 5Y Historical Beta (mean: 0.80, std: 0.074) <br>



In [22]:
stock_prices = reader.get_data_yahoo(ticker, start, end)['Adj Close']
stock_prices = stock_prices.resample('1M').last()
curr_price=stock_prices[ticker[0]][-1]
FiveYear_GR= gr_yearly.tail(5)
ThreeYear_GR=gr_yearly.tail(3)
g1=FiveYear_GR.mean()
g2=ThreeYear_GR.mean()
g3=0.02
n_iterations=20000
beta=[0.90,0.82,0.75,0.74] #beta values last 6 years

def multi_ddm(g1,g2,g3,div,cost_capital):
    d1=div*(1+g1)
    d2=div*(1+g1)**2
    d3=div*(1+g1)**3
    d4=d3*(1+g2)**1
    d5=d3*(1+g2)**2
    v5=(d5*(1+g3))/(cost_capital-g3)
    fair_price =  npf.npv(cost_capital,[d1, d2, d3, d4, d5+v5])
    return fair_price
results = []

g1_mean=gr_yearly.tail(10).mean()
g1_std=st.stdev(gr_yearly.tail(10))
g2_mean=gr_yearly.tail(5).mean()
g2_std=st.stdev(gr_yearly.tail(5))
g3_mean=0.02
g3_std=0.005
beta_mean=np.mean(beta)
beta_std=st.stdev(beta)

def multi_ddm_sim(div, beta_mean,beta_std , g1_mean, g1_std,g2_mean,g2_std,n_iterations):
    for i in range(n_iterations):
        
        beta_m =random.normalvariate(beta_mean, beta_std)
        g1 = random.normalvariate(g1_mean, g1_std)
        g2 = random.normalvariate(g2_mean, g2_std)
        g3 = random.normalvariate(g3_mean, g3_std)
        cost_capital=(risk_free + beta_m*market_premium + beta_s*size_premium + beta_v*value_premium)*12

        
        if g1 and g2 and g3 > (cost_capital-0.005):
            continue
        price=multi_ddm(g1,g2,g3,div,cost_capital)
        upside=((price-curr_price)/(curr_price))
        results.append((g1,g2,g3,cost_capital,beta_m,price,upside))               
    return results

def styled_df(df):
    return df.style.format({
        'High Growth': '{:.2%}',
        'Transitional Growth': '{:.2%}',
        'Perpetual Growth': '{:.2%}',
        'Cost of Capital': '{:.2%}',
        'Beta': '{:.2}',
        'Price' : '${:,.2f}',
        'Upside': '{:.2%}',

            }).background_gradient(cmap='RdYlGn', subset=['Price','Upside'])

results = multi_ddm_sim(div, beta_mean,beta_std , g1_mean, g1_std,g2_mean,g2_std,n_iterations)
df = pd.DataFrame(results, columns=['High Growth','Transitional Growth','Perpetual Growth','Cost of Capital','Beta', 'Price','Upside'])


In [23]:
fig = px.histogram(df, x="Price",nbins=1000,title="Probability Distribution")
fig.show()
#py.plot(fig, filename = 'dist', auto_open=True)

In [11]:
quants = df.quantile([i / 20 for i in range(1, 20)])
styled_df(quants)

,High Growth,Transitional Growth,Perpetual Growth,Cost of Capital,Beta,Price,Upside
0.050000,-3.37%,0.94%,1.17%,9.13%,0.68,$42.95,-22.61%
0.100000,-0.24%,2.28%,1.36%,9.34%,0.71,$47.18,-14.99%
0.150000,1.92%,3.21%,1.48%,9.48%,0.73,$50.30,-9.37%
0.200000,3.57%,3.96%,1.57%,9.60%,0.74,$52.91,-4.67%
0.250000,4.96%,4.55%,1.66%,9.70%,0.75,$55.11,-0.71%
0.300000,6.39%,5.14%,1.74%,9.79%,0.76,$57.20,3.06%
0.350000,7.55%,5.69%,1.80%,9.88%,0.77,$59.19,6.65%
0.400000,8.68%,6.20%,1.87%,9.96%,0.78,$61.14,10.16%
0.450000,9.75%,6.72%,1.94%,10.03%,0.79,$62.97,13.47%
0.500000,10.79%,7.22%,2.00%,10.10%,0.8,$64.75,16.68%


In [12]:
input_cols = [
    'High Growth', 
    'Transitional Growth',
    'Beta',
    'Cost of Capital',]


In [13]:
    fig = px.scatter(df, y='Price', x=input_cols[3],title="Cost of Capital Sensitivity")
    fig.show()

In [14]:
    fig = px.scatter(df, y='Price', x=input_cols[2],title="Beta Sensitivity")
    fig.show()

In [15]:
    fig = px.scatter(df, y='Price', x=input_cols[0],title="High Growth Sensitivity")
    fig.show()